# Summary tutorial: Getting metrics out of your models

The goal of the `learned_optimization.summary` module is to seamlessly allow researchers to annotate and extract data from *within* a jax computation / machine learning model.
This could be anything from mean and standard deviation of an activation, to looking at the distribution of outputs.

Doing this in Jax can be challenging at times as code written in Jax can make use of a large number of function transformations making it difficult to reach in and look at a value.

This notebook discusses the `learned_optimization.summary` module which provides one solution to this problem and is discussed in this notebook.

## Deps

In addition to `learned_optimization`, the summary module requires `oryx`. This can be a bit finicky to install at the moment as it relies upon particular versions of tensorflow (even though we never use these pieces).
All of `learned_optimization` will run without `oryx`, but to get summaries this module must be installed.
In a colab this is even more annoying as we must first upgrade the versions of some installed modules, restart the colab kernel, and then proceed to run the remainder of the cells.

In [ ]:
!pip install --upgrade git+https://github.com/google/learned_optimization.git oryx tensorflow==2.8.0rc0 numpy

  Cloning https://github.com/google/learned_optimization.git to /tmp/pip-req-build-vyc4pif3
  Running command git clone -q https://github.com/google/learned_optimization.git /tmp/pip-req-build-vyc4pif3
     |████████████████████████████████| 208 kB 5.2 MB/s 
     |████████████████████████████████| 492.1 MB 15 kB/s 
     |████████████████████████████████| 15.7 MB 40.3 MB/s 
     |████████████████████████████████| 154 kB 25.0 MB/s 
     |████████████████████████████████| 3.8 MB 33.9 MB/s 
     |████████████████████████████████| 179 kB 65.4 MB/s 
     |████████████████████████████████| 287 kB 71.0 MB/s 
     |████████████████████████████████| 126 kB 73.8 MB/s 
     |████████████████████████████████| 4.0 MB 56.5 MB/s 
     |████████████████████████████████| 1.4 MB 46.6 MB/s 
     |████████████████████████████████| 462 kB 73.4 MB/s 
     |████████████████████████████████| 65 kB 3.0 MB/s 
     |████████████████████████████████| 850 kB 58.4 MB/s 
     |████████████████████████████████| 6.0 MB

To check that everything is operating as expected we can check that the imports succeed.

In [ ]:
import oryx
from learned_optimization import summary
assert summary.ORYX_LOGGING

## Basic Example
Let's say that we have the following function and we wish to look at the `to_look_at` value.

In [ ]:
import jax
import jax.numpy as jnp

In [ ]:
def forward(params):
  to_look_at = jnp.mean(params) * 2.
  return params


def loss(parameters):
  loss = jnp.mean(forward(parameters)**2)
  return loss


value_grad_fn = jax.jit(jax.value_and_grad(loss))
value_grad_fn(1.0)

(DeviceArray(1., dtype=float32),
 DeviceArray(2., dtype=float32, weak_type=True))

With the summary module we can first annotate this value with `summary.summary`

In [ ]:
def forward(params):
  to_look_at = jnp.mean(params) * 2.
  summary.summary("to_look_at", to_look_at)
  return params


@jax.jit
def loss(parameters):
  loss = jnp.mean(forward(parameters)**2)
  return loss

Then we can transform the `loss` function with the function transformation: `summary.with_summary_output_reduced`.
This transformation goes through the computation and extracts all the tagged values and returns them to us by name in a dictionary.
In implementation, all the hard work here is done by the wonderful `oryx` library (in particular [harvest](https://github.com/tensorflow/probability/blob/main/spinoffs/oryx/oryx/core/interpreters/harvest.py)).
When we wrap a function this, we return a tuple containing the original result, and a dictionary with the desired metrics.

In [ ]:
result, metrics = summary.with_summary_output_reduced(loss)(1.)
result, metrics

(DeviceArray(1., dtype=float32),
 {'mean||to_look_at': DeviceArray(2., dtype=float32)})

As currently returned, the dictionary contains extra information as well as potentially duplicate values. We can collapse these metrics into a single value with the following:

In [ ]:
summary.aggregate_metric_list([metrics])

{'mean||to_look_at': 2.0}

The keys of this dictionary first show how the metric was aggregated. In this case there is only a single metric so the aggregation is ignored. This is followed by `||` and then the summary name.

One benefit of this function transformation is it can be nested with other jax function transformations. For example we can jit the transformed function like so:

In [ ]:
result, metrics = jax.jit(summary.with_summary_output_reduced(loss))(1.)
summary.aggregate_metric_list([metrics])

{'mean||to_look_at': 2.0}

At this point `aggregate_metric_list` cannot be jit. In practice this is fine as it performs very little computation.

## Aggregation of the same name summaries.

Consider the following fake function which calls a `layer` function twice.
This `layer` function creates a summary and thus two summary are created.
When running the transformed function we see not one, but two values returned.

In [ ]:
def layer(params):
  to_look_at = jnp.mean(params) * 2.
  summary.summary("to_look_at", to_look_at)
  return params * 2


@jax.jit
def loss(parameters):
  loss = jnp.mean(layer(layer(parameters))**2)
  return loss


result, metrics = summary.with_summary_output_reduced(loss)(1.)
result, metrics

(DeviceArray(16., dtype=float32),
 {'mean||to_look_at___2': DeviceArray(2., dtype=float32),
  'mean||to_look_at___3': DeviceArray(4., dtype=float32)})

These values can be combined with `aggregate_metric_list` as before, but this time the aggregation takes the mean. This `mean` is specified by the `aggregation` keyword argument in `summary.summary` which defaults to `mean`.

In [ ]:
summary.aggregate_metric_list([metrics])

{'mean||to_look_at': 3.0}

Another useful aggregation mode is "sample". As jax's random numbers are stateless, an additional RNG key must be passed in for this to work.

In [ ]:
def layer(params):
  to_look_at = jnp.mean(params) * 2.
  summary.summary("to_look_at", to_look_at, aggregation="sample")
  return params * 2


@jax.jit
def loss(parameters):
  loss = jnp.mean(layer(layer(parameters))**2)
  return loss


key = jax.random.PRNGKey(0)
result, metrics = summary.with_summary_output_reduced(loss)(
    1., sample_rng_key=key)
summary.aggregate_metric_list([metrics])

{'sample||to_look_at': 4.0}

Finally, there is `"collect"` which concatenates all the values together into one long tensor after first raveling all the inputs. This is useful for extracting distributions of quantities.

In [ ]:
def layer(params):
  to_look_at = jnp.mean(params) * 2.
  summary.summary(
      "to_look_at", jnp.arange(10) * to_look_at, aggregation="collect")
  return params * 2


@jax.jit
def loss(parameters):
  loss = jnp.mean(layer(layer(parameters))**2)
  return loss


key = jax.random.PRNGKey(0)
result, metrics = summary.with_summary_output_reduced(loss)(
    1., sample_rng_key=key)
summary.aggregate_metric_list([metrics])

{'collect||to_look_at': array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18.,  0.,  4.,  8.,
        12., 16., 20., 24., 28., 32., 36.], dtype=float32)}

## Summary Scope
Sometimes it is useful to be able to group all summaries from a function inside some code block with some common name. This can be done with the `summary_scope` context.

In [ ]:
@jax.jit
def loss(parameters):
  with summary.summary_scope("scope1"):
    summary.summary("to_look_at", parameters)

  with summary.summary_scope("nested"):
    summary.summary("summary2", parameters)

    with summary.summary_scope("scope2"):
      summary.summary("to_look_at", parameters)
  return parameters


key = jax.random.PRNGKey(0)
result, metrics = summary.with_summary_output_reduced(loss)(
    1., sample_rng_key=key)
summary.aggregate_metric_list([metrics])

{'mean||nested/scope2/to_look_at': 1.0,
 'mean||nested/summary2': 1.0,
 'mean||scope1/to_look_at': 1.0}

## Usage with function transforms.
Thanks to `oryx`, all of this functionality works well across a variety of function transformations.
Here is an example with a scan, vmap, and jit.
The aggregation modes will aggregate across all timesteps, and all batched dimensions.

In [ ]:
@jax.jit
def fn(a):
  summary.summary("other_val", a[2])

  def update(state, _):
    s = state + 1
    summary.summary("mean_loop", s[0])
    summary.summary("collect_loop", s[0], aggregation="collect")
    return s, s

  a, _ = jax.lax.scan(update, a, jnp.arange(20))
  return a * 2


vmap_fn = jax.vmap(fn)

result, metrics = jax.jit(summary.with_summary_output_reduced(vmap_fn))(
    jnp.tile(jnp.arange(4), (2, 2)))
summary.aggregate_metric_list([metrics])

{'collect||collect_loop': array([ 1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,
         9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17,
        18, 18, 19, 19, 20, 20], dtype=int32),
 'mean||mean_loop': 10.5,
 'mean||other_val': 2.0}

## Optionally compute metrics: `@add_with_metrics`

Oftentimes it is useful to define two "versions" of a function -- one with metrics, and one without -- as sometimes the computation of the metrics adds unneeded overhead that does not need to be run every iteration.
To create these two versions one can simply wrap the function with the `add_with_summary` decorator.
This adds both a keyword argument, and an extra return to the wrapped function which switches between computing metrics, or not.

In [ ]:
from learned_optimization import summary
import functools


def layer(params):
  to_look_at = jnp.mean(params) * 2.
  summary.summary("to_look_at", jnp.arange(10) * to_look_at)
  return params * 2


@functools.partial(jax.jit, static_argnames="with_summary")
@summary.add_with_summary
def loss(parameters):
  loss = jnp.mean(layer(layer(parameters))**2)
  return loss


res, metrics = loss(1., with_summary=False)
print("No metrics", summary.aggregate_metric_list([metrics]))

res, metrics = loss(1., with_summary=True)
print("With metrics", summary.aggregate_metric_list([metrics]))

No metrics {}
With metrics {'mean||to_look_at': 13.5}


## Limitations and Gotchas

### Requires value traced to be a function of input

At the moment summary.summary MUST be called with a descendant of whatever input is passed into the function wrapped by `with_summary_output_reduced`. In practice this is almost always the case as we seek to monitor changing values rather than constants.

To demonstrate this, note how the constant value is NOT logged out, but if add it to `a*0` it does become logged out.

In [ ]:
def monitor(a):
  summary.summary("with_input", a)
  summary.summary("constant", 2.0)
  summary.summary("constant_with_inp", 2.0 + (a * 0))
  return a


result, metrics = summary.with_summary_output_reduced(monitor)(1.)
summary.aggregate_metric_list([metrics])

{'mean||constant_with_inp': 2.0, 'mean||with_input': 1.0}

The rational for why this is a bit of a rabbit hole, but it is related to how tracing in jax work and is beyond the scope of this notebook.

### No support for jax.lax.cond

At this point one cannot extract summaries out of jax conditionals. Sorry. If this is a limitation to you let us know as we have some ideas to make this work.

### No dynamic names

At the moment, the tag, or the name of the summary, must be a string known at compile time. There is no support for dynamic summary names.

## Alternatives for extracting information
Using this module is not the only way to extract information from a model. We discuss a couple other approaches.

### "Thread" metrics through
One way to extract data from a function is to simply return the things we want to look at. As functions become more complex and nested this can become quite a pain as each one of these functions must pass out metric values. This process of spreading data throughout a bunch of functions is called "threading".

Threading also requires all pieces of code to be involved -- as one must thread these metrics everywhere.

In [ ]:
def lossb(p):
  to_look_at = jnp.mean(123.)
  return p * 2, to_look_at


def loss(parameters):
  l = jnp.mean(parameters**2)
  l, to_look_at = lossb(l)
  return l, to_look_at


value_grad_fn = jax.jit(jax.value_and_grad(loss, has_aux=True))
(loss, to_look_at), g = value_grad_fn(1.0)
print(to_look_at)

123.0


### jax.experimental.host_callback

Jax has some support to send data back from an accelerator back to the host while a ja program is running. This is exposed in jax.experimental.host_callback.

One can use this to print which is a quick way to get data out of a network.

In [ ]:
from jax.experimental import host_callback as hcb


def loss(parameters):
  loss = jnp.mean(parameters**2)
  to_look_at = jnp.mean(123.)
  hcb.id_print(to_look_at, name="to_look_at")
  return loss


value_grad_fn = jax.jit(jax.value_and_grad(loss))
_ = value_grad_fn(1.0)

name: to_look_at
123.


It is also possible to extract data out with [host_callback.id_tap](https://jax.readthedocs.io/en/latest/jax.experimental.host_callback.html#jax.experimental.host_callback.id_tap). We experimented with this briefly for a summary library but found both performance issues and increased complexity around custom transforms.